# Experiments on Graph-Generative-Models
In this notebook, we aim to evluate the performance of "GDSS" proposed in "Score-based Generative Modeling of Graphs via the System of Stochastic Differential Equations" (https://arxiv.org/pdf/2202.02514.pdf). The baseline model is tested on 3 datasets (Grid, Protein, 3D Point Cloud) and measured under 4 metrics (degree, clustering, orbit, graph spectrum).

### Experiment on GDSS
Here we immigrate the original terminal-executable GDSS codes into the notebook.

##### Change current directory.

In [1]:
import os
path = "./GDSS/"
os.chdir(path)

##### Install dependencies

In [ ]:
!pip install -r requirements.txt
!conda install -c conda-forge rdkit=2020.09.1.0
!yes | pip install git+https://github.com/fabriziocosta/EDeN.git --user!

##### Assign dataset and seed

In [2]:
dataset = 'ego_small'
seed = 42

##### Generate dataset

In [3]:
!python data/data_generators.py --dataset $dataset

ego_small 200
17


##### Train the GDSS model

In [3]:
import torch
import argparse
import time
from parsers.config import get_config
from trainer import Trainer
from sampler import Sampler, Sampler_mol


os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
torch.cuda.empty_cache()

ts = time.strftime('%b%d-%H:%M:%S', time.gmtime())
config = get_config(dataset, seed)
trainer = Trainer(config) 
ckpt = trainer.train(ts)
if 'sample' in config.keys():
    config.ckpt = ckpt
    sampler = Sampler(config) 
    sampler.sample()

----------------------------------------------------------------------------------------------------
Make Directory ego_small/test in Logs
Sep07-04:22:47
----------------------------------------------------------------------------------------------------
[ego_small]   init=deg (17)   seed=42   batch_size=128
----------------------------------------------------------------------------------------------------
lr=0.01 schedule=True ema=0.999 epochs=5000 reduce=False eps=1e-05
(ScoreNetworkX)+(ScoreNetworkA=GCN,4)   : depth=2 adim=32 nhid=32 layers=5 linears=2 c=(2 8 4)
(x:VP)=(0.10, 1.00) N=1000 (adj:VP)=(0.10, 1.00) N=1000
----------------------------------------------------------------------------------------------------


[Epoch]:  20%|██        | 1000/5000 [09:29<38:47,  1.72it/s]

[EPOCH 1000] test adj: 3.993e+00 | train adj: 4.592e+00 | test x: 5.648e+00 | train x: 5.793e+00


[Epoch]:  40%|████      | 2000/5000 [18:54<29:51,  1.67it/s]

[EPOCH 2000] test adj: 3.655e+00 | train adj: 3.734e+00 | test x: 4.639e+00 | train x: 4.467e+00


[Epoch]:  60%|██████    | 3000/5000 [28:17<18:52,  1.77it/s]

[EPOCH 3000] test adj: 4.411e+00 | train adj: 3.665e+00 | test x: 5.130e+00 | train x: 4.262e+00


[Epoch]:  62%|██████▏   | 3109/5000 [29:20<18:18,  1.72it/s]

##### Generate new graphs by the trained GDSS model

In [ ]:
if dataset in ['QM9', 'ZINC250k']:
    sampler = Sampler_mol(config)
else:
    sampler = Sampler(config) 
sampler.sample()